## PhoREAL / SlideRule Example

## Background
### PhoREAL

ICESat-2 is a space-based laser altimetry mission that provides accurate 3-D representations of the earth’s surface. The Advanced Topographic Laser Altimeter System (ATLAS) onboard ICESat-2 can measure surface heights with great accuracy, providing critical measurements needed to better understand key surface structure variables. The Photon Research and Analysis Library (PhoREAL) was designed to provide a customizable analysis tool for NASA’s ICESat-2 Land and Vegetation (ATL08) data (https://github.com/icesat-2UT/PhoREAL). With PhoREAL, users can resample, reproject, and recalculate terrain and canopy height statistics at any along-track resolution.

### ATL06
The ATL06 data set (https://nsidc.org/data/atl06/versions/6) provides geolocated, land-ice surface heights (above the WGS 84 ellipsoid, ITRF2014 reference frame), plus ancillary parameters that can be used to interpret and assess the quality of the height estimates. The data were also acquired by the ATLAS instrument on board ICESat-2. ATL06-SR refers to ATL06-SlideRule products.

## The goal of this demo
Demonstrate running the PhoREAL algorithm in SlideRule to produce canopy metrics over the Grand Mesa, Colorado region.

#### Imports

In [ ]:
# Suppress warnings
import warnings
warnings.filterwarnings("ignore") 

In [ ]:
# Import requisite packages
import matplotlib.pyplot as plt
import matplotlib
import geopandas
import logging
import sliderule
from sliderule import icesat2

#### Initialize SlideRule Client

In [ ]:
icesat2.init("slideruleearth.io", verbose=True, loglevel=logging.INFO)

#### Processing parameters (https://slideruleearth.io/web/rtd/user_guide/icesat2.html#parameters)
* 100 m segments stepped every 100 m
* Subsetted to the Grand Mesa region
* Time range: One day (Nov 14, 2019)
* Only processing ground, canopy, and top of canopy photons
* Request the "h_dif_ref" variable as an ancillary field to be included in the results
* Running PhoREAL algorithm using a binsize of 1 m, and geolocating each segment at the center of the segment
* Sending reconstructed waveforms along with metrics (for diagnostics and demonstration purposes only)

In [ ]:
parms = {
    "poly": sliderule.toregion('grandmesa.geojson')['poly'], # polygon defining region of interest
    "t0": '2019-11-14T00:00:00Z', # start time for filtering granules (format %Y-%m-%dT%H:%M:%SZ)
    "t1": '2019-11-15T00:00:00Z', # stop time for filtering granules (format %Y-%m-%dT%H:%M:%SZ)
    "srt": icesat2.SRT_LAND, # surface type: 0-land, 1-ocean, 2-sea ice, 3-land ice, 4-inland water
    "len": 100, # length of each extent in meters
    "res": 100, # step distance for successive extents in meters
    "pass_invalid": True, # true|false flag indicating whether or not extents that fail validation checks are still used and returned in the results
    "atl08_class": ["atl08_ground", "atl08_canopy", "atl08_top_of_canopy"],
    "atl08_fields": ["h_dif_ref"],
    "phoreal": {"binsize": 1.0, "geoloc": "center", "use_abs_h": False, "send_waveform": True}
}

#### Make ATL08 Request

In [ ]:
atl08 = icesat2.atl08p(parms, keep_id=True)

#### Print Resulting GeoDataFrame

In [ ]:
atl08

#### Plot Canopy Height

In [ ]:
canopy_gt1l = atl08[atl08['gt'] == icesat2.GT1L]
ax = canopy_gt1l.plot.scatter(x='x_atc', y='h_canopy')
ax.set_xlabel('Along-track Distance (m) ')
ax.set_ylabel('Canopy Height (m) ')    

#### Plot Landcover

In [ ]:
fig, ax1 = plt.subplots(figsize=(9, 6))  

# Use ax1 instead of ax for consistency
h_sc = atl08.plot(column='landcover', ax=ax1)

ax1.set_xlabel('Longitude (°)') 
ax1.set_ylabel('Latitude (°)')     
ax1.set_aspect('auto')  

# Add horizontal colorbar
cbar = fig.colorbar(h_sc.collections[0], ax=ax1, orientation='horizontal', label='Land Cover')

#### Create and Plot 75th percentile Across All Ground Tracks 
We are plotting the 75th percentile of the distribution of canopy height photons, which is a standard measure of tree heights.

In [ ]:
atl08['75'] = atl08.apply(lambda row : row["canopy_h_metrics"][icesat2.P['75']], axis = 1)
ax = atl08.plot.scatter(x='x_atc', y='75')
ax.set_xlabel('Along-track Distance (m) ')
ax.set_ylabel('Canopy Height (m) ') 

#### Create Sample Waveform Plots
These are histograms showing photon heights

In [ ]:
num_plots = 5
waveform_index = [96, 97, 98, 100, 101]
fig,ax = plt.subplots(num=1, ncols=num_plots, sharey=True, figsize=(12, 6))
for x in range(num_plots):
    ax[x].plot([x for x in range(len(canopy_gt1l['waveform'][waveform_index[x]]))], canopy_gt1l['waveform'][waveform_index[x]], zorder=1, linewidth=1.0, color='mediumseagreen')
plt.show()

#### Make ATL06-SlideRule (ATL06-SR) Request
* Now we will run an ATL06-SR processing request on the same source data using the same parameters.  Because the `keep_id` argument is set to true here and above when we made the ATL08 request, we can merge the resulting dataframes and have a single table of: (1) elevation data using the customized ATL06-SR algorithm, and (2) vegatation data using the PhoREAL algorithm.

In [ ]:
atl06 = icesat2.atl06p(parms, keep_id=True)

#### Merge Atl06 and Atl08 GeoDataFrames

In [ ]:
gdf = geopandas.pd.merge(atl08, atl06, on='extent_id', how='left', suffixes=('.atl08','.atl06')).set_axis(atl08.index)
gdf